In [ ]:
import sys 
import gymnasium as gym
import kidspuzzles
from stable_baselines3 import A2C

env = gym.make('kidspuzzles/DigitsPuzzleEnv-v0', size = 10, render_mode = 'human')

model = A2C("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=10)

vec_env = model.get_env()
observations = vec_env.reset()
for _ in range(100):
    action, _state = model.predict(observations, deterministic=True)
    observations, reward, terminated, info = vec_env.step(action)
    vec_env.render("human")
    # if terminated:
    #     observations, info = vec_env.reset()

env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


: 